In [ ]:
import pickle

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
from infer.aux import genData, processTrace, updateRes, Reporter, mutate_data, mutate_data2
from infer.model import Model

In [ ]:
Xgt, Y, rels = genData(60, 10, 20000)

In [ ]:
pickle.dump(Xgt, open("v6-copy0-Xgt.p", "wb"))
pickle.dump(rels, open("v6-copy0-rels.p", "wb"))

In [ ]:
# this is for adding noise to input data
#Y = mutate_data2(Y, 0.05)

In [ ]:
alph, beta = 0.5, 1.
x_alph = np.ones(shape=len(Xgt))*alph
x_beta = np.ones(shape=len(Xgt))*beta

r_alph = np.ones(shape=len(rels))*alph
r_beta = np.ones(shape=len(rels))*beta

alph = beta = 0.7
s_alph = np.ones(shape=len(rels))*alph
s_beta = np.ones(shape=len(rels))*beta

Xprior = st.beta(x_alph, x_beta)
Rprior = st.beta(r_alph, r_beta)
Sprior = st.beta(s_alph, s_beta)

In [ ]:
njobs = 2
chains = 2

model = Model()
model.scale = [0.3]
model.build(Y, rels, [Xprior, Rprior, Sprior])

model.set_subsample_size(0.001)

model.init_chains(chains=chains)

In [ ]:
reporter = Reporter()
reporter.report("Start")
while not model.converged():
    model.sample(N=50000, burn=0.3, thin=5000, njobs=njobs)
    pickle.dump(model, open("v6-copy0-model.p", "wb"))
    reporter.report("Completed current sample")

In [ ]:
model.chains[0].scale = 0.15
model.chains[1].scale = 0.15
model.scale[0] = 0.15

In [ ]:
reporter = Reporter()
reporter.report("Start")
while not model.converged():
    model.sample(N=100, burn=0, thin=5, njobs=njobs)
    pickle.dump(model, open("v6-copy0-model.p", "wb"))
    reporter.report("Completed current sample")

In [ ]:
pickle.dump(model, open("v6-copy0-model.p", "wb"))

In [ ]:
grX, grR, grS = model.gelman_rubin.values()
grX.max(), grR.max(), grS.max()

In [ ]:
y=np.array([model.trace[0]['R'][:,270949], model.trace[1]['R'][:,270949]])

In [ ]:
y.shape

In [ ]:
plt.plot(y.T)

In [ ]:
np.argwhere(grR>1e16)

In [ ]:
model.trace[0]['X'].shape

In [ ]:
y = model.get_trace('X', chain=0)
for i in range(y.shape[1]):
    plt.plot(range(y.shape[0]), y[:, i], alpha=0.4)
plt.ylim(0,1)
plt.show()

In [ ]:
Xres, Rres, Sres = processTrace(model, Xgt, rels)
Xres, Rres, Sres = updateRes(Xres, Rres, Sres, final=True)

In [ ]:
Xres.gelman_rubin.max(), Rres.gelman_rubin.max(), Sres.gelman_rubin.max()

In [ ]:
h = plt.hist(Xres.gelman_rubin, 48, (0.9, 1.5))
plt.vlines(1, 0, h[0].max(), colors='r')

In [ ]:
h = plt.hist(Rres.gelman_rubin, 48, (0.9, 1.5))
plt.vlines(1, 0, h[0].max(), colors='r')

In [ ]:
h = plt.hist(Sres.gelman_rubin, 48, (0.9, 1.5))
plt.vlines(1, 0, h[0].max(), colors='r')

In [ ]:
Xres[~Xres.gelman_rubin.between(0.95,1.05)]

In [ ]:
Rres[~Rres.gelman_rubin.between(0.95,1.05)]

In [ ]:
Sres[~Sres.gelman_rubin.between(0.95,1.05)]

In [ ]:
Xres[~Xres.correctness]

In [ ]:
Rres[~Rres.correctness]

In [ ]:
Sres[~Sres.correctness]

In [ ]:
plt.ylim(0,1)
plt.plot(range(len(Rres)), Rres['mean'])

In [ ]:
Xres

In [ ]:
Rres

In [ ]:
Sres

In [ ]:
tmp = Rres[Rres.gelman_rubin.between(0.99,1.01)].correctness
tmp.sum()/tmp.size

In [ ]:
tmp = Rres[Rres.gelman_rubin.between(0.95,1.05)].correctness
tmp.sum()/tmp.size

In [ ]:
tmp = Rres.correctness
tmp.sum()/tmp.size

In [ ]:
tmp.size-tmp.sum()